3/7/2024 AMD
I used Claude 3.0 to write a script to enable us to input multiple cubes with multiple images (each cube taken with a single exposure time, multiple filter sets) and process the different exposure times to generate a single HDR image.

In [2]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
import xarray as xr

def aggregate_exposures(images, exposure_times):
    # Convert exposure times to relative radiance values
    radiance_values = np.log(exposure_times)
    
    # Initialize the HDR image
    hdr_image = np.zeros(images[0].shape, dtype=np.float32)
    
    # Iterate over each pixel in the images
    for i in range(images[0].shape[0]):
        for j in range(images[0].shape[1]):
            # Collect pixel values from all images
            pixel_values = np.array([img[i, j] for img in images], dtype=np.float32)
            
            # Perform weighted average based on radiance values
            weights = np.exp(radiance_values - np.max(radiance_values))
            weighted_sum = np.sum(weights * pixel_values)
            weight_sum = np.sum(weights)
            
            # Assign the aggregated value to the HDR image
            hdr_image[i, j] = weighted_sum / weight_sum
    
    return hdr_image

# User prompts for input
num_filters = int(input("Enter the number of filters: "))

image_files = []
exposure_times = []

for i in range(num_filters):
    image_file = input(f"Enter the path to the image h5 file for filter {i+1}: ")
    exposure_time = float(input(f"Enter the exposure time for filter {i+1} in seconds: "))
    image_files.append(image_file)
    exposure_times.append(exposure_time)

# Initialize the HDR image cube
with h5py.File(image_files[0], 'r') as f:
    image_shape = f['image_0'].shape
    num_images = len(f.keys())

hdr_image_cube = np.zeros((num_filters, image_shape[0], image_shape[1]), dtype=np.float32)

# Process each filter
for i in range(num_filters):
    # Extract images from the h5 file
    with h5py.File(image_files[i], 'r') as f:
        images = [np.array(f['image_{}'.format(j)]) for j in range(num_images)]
    
    # Display the original images for the current filter
    fig, axs = plt.subplots(1, num_images, figsize=(15, 5))
    for j, img in enumerate(images):
        axs[j].imshow(img, cmap='gray')
        axs[j].set_title(f'Filter {i+1}, Image {j+1}')
        axs[j].axis('off')
    plt.tight_layout()
    plt.show()
    
    # Aggregate pixel values from multiple exposures for the current filter
    hdr_image = aggregate_exposures(images, [exposure_times[i]] * num_images)
    
    # Normalize the HDR image for display
    hdr_image_normalized = (hdr_image - np.min(hdr_image)) / (np.max(hdr_image) - np.min(hdr_image))
    hdr_image_normalized = (hdr_image_normalized * 65535).astype(np.uint16)
    
    # Assign the HDR image to the corresponding slice in the HDR image cube
    hdr_image_cube[i] = hdr_image_normalized
    
    # Display the normalized HDR image for the current filter
    plt.figure(figsize=(8, 8))
    plt.imshow(hdr_image_normalized, cmap='gray')
    plt.title(f'Normalized HDR Image - Filter {i+1}')
    plt.axis('off')
    plt.show()

# Save the HDR image cube
output_file = input("Enter the output file name for the HDR image cube (e.g., hdr_image_cube.npy): ")
np.save(output_file, hdr_image_cube)

ValueError: could not convert string to float: ''